# Training YOLOv8 for Drowsiness Detection

This notebook uses the Simuletic_DMS_Dataset to train a YOLOv8 classification model for detecting drowsiness based on eye state.

In [ ]:
import os
import json
import shutil
import random
from ultralytics import YOLO
import matplotlib.pyplot as plt

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Update ultralytics (uncomment if needed)
# %pip install -U ultralytics


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Fix torch installation (uncomment if needed)
# %pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.9.1+cpu
Uninstalling torch-2.9.1+cpu:
  Successfully uninstalled torch-2.9.1+cpu
Found existing installation: torchvision 0.24.1+cpu
Uninstalling torchvision-0.24.1+cpu:
  Successfully uninstalled torchvision-0.24.1+cpu
Found existing installation: torchaudio 2.9.1+cpu
Uninstalling torchaudio-2.9.1+cpu:
  Successfully uninstalled torchaudio-2.9.1+cpu
Note: you may need to restart the kernel to use updated packages.


In [22]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.9.1%2Bcpu-cp312-cp312-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.24.1%2Bcpu-cp312-cp312-win_amd64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.9.1%2Bcpu-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.9.1%2Bcpu-cp312-cp312-win_amd64.whl (110.9 MB)
Using cached https://download.pytorch.org/whl/cpu/torchvision-0.24.1%2Bcpu-cp312-cp312-win_amd64.whl (4.3 MB)
Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.9.1%2Bcpu-cp312-cp312-win_amd64.whl (663 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Prepare dataset
# Define paths
dataset_dir = 'Simuletic_DMS_Dataset'
images_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')

# Create initial directories for classes
train_dir = 'train_data'
awake_dir = os.path.join(train_dir, 'awake')
drowsy_dir = os.path.join(train_dir, 'drowsy')
os.makedirs(awake_dir, exist_ok=True)
os.makedirs(drowsy_dir, exist_ok=True)

# Load labels and copy images based on eye_state
for label_file in os.listdir(labels_dir):
    if label_file.endswith('.json'):
        with open(os.path.join(labels_dir, label_file), 'r') as f:
            data = json.load(f)
            for sample in data['samples']:
                image_name = sample['image']
                eye_state = sample['attributes']['eye_state']
                src = os.path.join(images_dir, image_name)
                if eye_state == 'Open':
                    dst = os.path.join(awake_dir, image_name)
                elif eye_state == 'Closed':
                    dst = os.path.join(drowsy_dir, image_name)
                else:
                    continue
                if os.path.exists(src):
                    shutil.copy(src, dst)

print("Dataset prepared.")

FileNotFoundError: [Errno 2] No such file or directory: '/Simuletic_DMS_Dataset/labels'

In [27]:
# Split into train and val sets
random.seed(42)  # For reproducibility

# Define subdirectories for train and val
train_awake_dir = os.path.join(train_dir, 'train', 'awake')
train_drowsy_dir = os.path.join(train_dir, 'train', 'drowsy')
val_awake_dir = os.path.join(train_dir, 'val', 'awake')
val_drowsy_dir = os.path.join(train_dir, 'val', 'drowsy')

# Create directories
for dir_path in [train_awake_dir, train_drowsy_dir, val_awake_dir, val_drowsy_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Split each class: 80% train, 20% val
for class_dir, train_class_dir, val_class_dir in [(awake_dir, train_awake_dir, val_awake_dir), (drowsy_dir, train_drowsy_dir, val_drowsy_dir)]:
    images = os.listdir(class_dir)
    random.shuffle(images)
    split_idx = int(0.8 * len(images))
    train_images = images[:split_idx]
    val_images = images[split_idx:]
    for img in train_images:
        shutil.move(os.path.join(class_dir, img), os.path.join(train_class_dir, img))
    for img in val_images:
        shutil.move(os.path.join(class_dir, img), os.path.join(val_class_dir, img))
    os.rmdir(class_dir)  # Remove empty original dir

print("Dataset split into train and val.")

Dataset split into train and val.


In [ ]:
# Train YOLOv8 classification model
# Create a new YOLO model for classification
model = YOLO()  # Defaults to classification if task specified
# Train with specified parameters
results = model.train(task='classify', data=train_dir, epochs=10, imgsz=224)

print("Training completed.")

New https://pypi.org/project/ultralytics/8.3.232 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.199  Python-3.12.5 torch-2.7.0+cpu CPU (Intel Core i5-5300U 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=train_data, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False,

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
# Display training results
print("Training Results:")
if hasattr(results, 'results_dict') and results.results_dict:
    metrics = results.results_dict
    print(f"Top-1 Accuracy: {metrics.get('metrics/accuracy_top1', 'N/A')}")
    print(f"Top-5 Accuracy: {metrics.get('metrics/accuracy_top5', 'N/A')}")
    print(f"Precision: {metrics.get('metrics/precision(B)', 'N/A')}")
    print(f"Recall: {metrics.get('metrics/recall(B)', 'N/A')}")
    print(f"F1 Score: {metrics.get('metrics/f1(B)', 'N/A')}")
else:
    print("Metrics not available. Check results object.")

# Plot results if available
try:
    if hasattr(results, 'plots'):
        results.plots()
except Exception as e:
    print(f"Plotting failed: {e}")

Training Results:


NameError: name 'results' is not defined

In [ ]:
# Save the trained model
model.save('drowsiness_model.pt')
print("Model saved as drowsiness_model.pt")

In [ ]:
# Test the model on sample images
# Test on awake image
test_image_awake = os.path.join(train_awake_dir, os.listdir(train_awake_dir)[0])
results_awake = model.predict(test_image_awake)
print(f"Prediction for awake image: {results_awake[0].probs.top1} (confidence {results_awake[0].probs.top1conf:.2f})")

# Test on drowsy image
test_image_drowsy = os.path.join(train_drowsy_dir, os.listdir(train_drowsy_dir)[0])
results_drowsy = model.predict(test_image_drowsy)
print(f"Prediction for drowsy image: {results_drowsy[0].probs.top1} (confidence {results_drowsy[0].probs.top1conf:.2f})")

In [ ]:
# Test the model on awake image
test_image_awake = os.path.join(train_awake_dir, os.listdir(train_awake_dir)[0])
results_awake = model.predict(test_image_awake)
print(f"Prediction for awake image: {results_awake[0].probs.top1} (class {results_awake[0].probs.top1conf:.2f})")

# Test on drowsy image
test_image_drowsy = os.path.join(train_drowsy_dir, os.listdir(train_drowsy_dir)[0])
results_drowsy = model.predict(test_image_drowsy)
print(f"Prediction for drowsy image: {results_drowsy[0].probs.top1} (class {results_drowsy[0].probs.top1conf:.2f})")